# VICAV Bibliografie: Zotero Update

In [ ]:
!ls

## Parse TEI File and extract xml:ids and zotero-uris

In [ ]:
tei_bibl_filename = "vicav_biblio_tei_zotero.xml"

In [ ]:
#import Package eTree to parse XML Files
import xml.etree.ElementTree as ET

In [ ]:
tei_bibl_file = ET.parse(tei_bibl_filename)
TEI = tei_bibl_file.getroot()

xmlns = {"tei": "http://www.tei-c.org/ns/1.0", "xml":"http://www.w3.org/XML/1998/namespace" }

In [ ]:
#/TEI/text[1]/body[1]/listBibl[1]/biblStruct[1]
bibls = TEI.findall("tei:text/tei:body/tei:listBibl/tei:biblStruct", xmlns)

In [ ]:
len(bibls)

In [ ]:
bibl_data = []
for bibl in bibls:
    item = {}
    item["zotero_url"] = bibl.attrib['corresp']
    item["xml_id"] = bibl.get('{http://www.w3.org/XML/1998/namespace}id')
    item["zotero_id"] = item["zotero_url"].split("/items/")[1]
    bibl_data.append(item)

In [ ]:
len(bibl_data)

In [ ]:
bibl_data[0]

## Access Zotero-API

In [ ]:
import requests
import json

In [ ]:
api_token = "DQfd2PYuoDza5S0zMQdjzU6P"

In [ ]:
#Group-id of Vicav library
group_id = "2165756"

In [ ]:
#function to get the value of the "extra" field
def get_extra_field(zotero_id):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + zotero_id
    response = requests.get(request_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        
        extra = parsed["data"]["extra"]
        
        if extra.startswith("(biblid:"):
            extra = extra.split("(biblid:")[1].split(")")[0]
    else: 
        extra = None        
    
    return extra

In [ ]:
#test
test_id = bibl_data[0]["zotero_id"]
get_extra_field(test_id)

In [16]:
for item in bibl_data:
    if "biblid" not in item:
        extra = get_extra_field(item["zotero_id"])
        if extra != "":
            item["biblid"] = extra

In [17]:
bibl_data[4]

{'zotero_url': 'http://zotero.org/groups/2165756/items/UX9BTLFV',
 'xml_id': 'Zellal1995',
 'zotero_id': 'UX9BTLFV',
 'biblid': 'zellal_1995_0000'}

In [18]:
missing = []
for item in bibl_data:
    if 'biblid' not in item:
        missing.append(item) 

In [ ]:
len(missing)

In [20]:
missing[0]

{'zotero_url': 'http://zotero.org/groups/2165756/items/7IUUFEWI',
 'xml_id': 'Lorimer',
 'zotero_id': '7IUUFEWI'}

In [21]:
with open('bibl_data.json', 'w') as outfile:
    json.dump(bibl_data, outfile)

# need to read again

In [22]:
#read backup file

with open('bibl_data.json', 'r') as infile:
    bibl_data = json.load(infile)

len(bibl_data)

4722

In [23]:
bibl_data[0]

{'zotero_url': 'http://zotero.org/groups/2165756/items/G2MMP7G2',
 'xml_id': 'Zimmermann1994',
 'zotero_id': 'G2MMP7G2',
 'biblid': 'zimmermann_1994_2986'}

In [24]:
missing = []
for item in bibl_data:
    if 'biblid' not in item:
        missing.append(item) 

In [28]:
len(missing)

6

In [27]:
missing[0]

{'zotero_url': 'http://zotero.org/groups/2165756/items/7IUUFEWI',
 'xml_id': 'Lorimer',
 'zotero_id': '7IUUFEWI'}

In [29]:
import re
import random

In [30]:
def id_to_biblid(xml_id:str):
    m = re.search('(\w*)([0-9]{4})(\w?)', xml_id)
    name = m.group(1)
    year = m.group(2)
    suffix = m.group(3)
    #print("Name: " + name + " Year: " + year + " Suffix: " + suffix )
    r = random.randint(1,9999)
    randnr = "%04d" % r
    new_id = name.lower() + "_" + year + "_" + randnr
    return new_id

In [31]:
#test
testid = missing[0]["xml_id"] 
id_to_biblid(testid)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
#uris MUST use https://

In [ ]:
#https://api.zotero.org/groups/2165756/items/G2MMP7G2

In [ ]:
api_token = "DQfd2PYuoDza5S0zMQdjzU6P"
headers = {'Authorization': 'Bearer ' + api_token}

In [ ]:
# Function to update zotero-item

def update_zotero_extra_in_item(group_id:str, item_id:str, extra:str):
    #test_url = "https://api.zotero.org/groups/4669401/items/P4TV9RFP"
    
    url = "https://api.zotero.org/groups/" + group_id + "/items/" + item_id
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        
        if "extra" in parsed["data"]:
            old_extra_val = parsed["data"]["extra"]
            #print("Existing 'extra' field value was: " + old_extra_val)
            
            #updating
            parsed["data"]["extra"] = extra
            
            update = requests.put(url, headers=headers, json=parsed)
            if update.status_code == 204:
                #everything worked
                #return True as idicator, that everything worked as expected, and the old "extra" value (could be stored somewhere for safe-keeping)
                return [True,old_extra_val]
            else:
                print("Error when updating item. Status code:" + str(update.status_code))
        
    else:
        print("Error when getting item. Status code: " + str(response.status_code))    
    

In [ ]:
missing[0]

In [53]:
#vicav group
group_id

'2165756'

In [74]:
#working example
testitem = missing[0]
testid = missing[0]["xml_id"]
item_id = testitem["zotero_id"]
new_id = id_to_biblid(testid)
new_extra = "(" + "biblid:" + new_id + ")"
print(new_extra)
#group_id
updated = update_zotero_extra_in_item(group_id, item_id, new_extra)
updated

(biblid:akin_2017_5979)


[True, '(biblid:ziagos_2018_0269)']

In [75]:
for item in missing:
    print("Updating " + item["xml_id"])
    try:
        new_id = id_to_biblid(item["xml_id"])
    except:
        new_id = None
        print("error to generate id: " + item["xml_id"])
    
    if new_id != None:
        new_extra = "(" + "biblid:" + new_id + ")"
        item["biblid"] = new_id
        updated = update_zotero_extra_in_item(group_id, item["zotero_id"], new_extra)
        item["updated"] = updated[0]
        item["old_extra_val"] = updated[1]

Updating Akin2017
Updating Seeger2021
Updating Ziamari2006a
Updating Zeroual2008
Updating Zeroual2006
Updating Zafrani1988a
Updating Zafrani1987
Updating Youssi2003a
Updating Youssi1994
Updating Youssi1984
Updating Youssi1980
Updating Vycichl1973b
Updating Vicente2008a
Updating Vicente2009a
Updating Vicente2002a
Updating Tamer2003
Updating Stillman1998
Updating Stempel2009
Updating Shuol2008
Updating Sánchez2009
Updating Saib1988
Updating Sadiqi2002b
Updating Sadiqi2002c
Updating Ruiter2009b
Updating Ruiter2008b
Updating Ruiter2006a
Updating Rosenhouse1996
Updating Retsö2014
Updating Ratcliffe2002
Updating Pat-El2014
Updating Nortier1991
Updating Ibn_Srift1994
Updating Mourchid1997
Updating Moscoso_García2009b
Updating Mohamed_Yeou2008
Updating Messaoudi2002
Updating Martínez_Ruiz1982
Updating Mahi2006
Updating Mahi2003
Updating Mahi2002
Updating Maas2003
Updating Maas2002
Updating Lévy2002
Updating Lévy1994a
Updating Lentin2006c
Updating Langone2006
Updating Lamrani2003
Updating Lamra

Updating Ait_Hamza1991
Updating Ahmed1986
Updating Aguadé2000b
Updating Aguadé1998e
Updating Aguadé1995b
Updating Agius1992
Updating Agius1990
Updating Abu-Mansour1992a
Updating Abu-Haidar2006
Updating Abu-Haidar1979
Updating Wende2000
Updating Aboul-Fetouh1969
Updating _Āl1968
Updating Fischer1991
Updating Fischer1967
Updating Fakhry2016
Updating Hanitsch2016
Updating Ferré1952
Updating Fernández1911
Updating Fenech1978
Updating Feghali1991
Updating Féghali1938
Updating Féghali1919
Updating Ess1938
Updating Erwin1969
Updating Haig2017
Updating Ennaji1988
Updating El-Zein1981
Updating El-Hassan1978
Updating Elgibali1993
Updating Elder1927
Updating Eisele1992
Updating Eisele1990a
Updating Eisele1990b
Updating Dwyer1987
Updating Durand1995c
Updating Drewes1994
Updating Dostal1983
Updating Doss1996
Updating Djebli1988
Updating Diem1974b
Updating Díaz1981
Updating Díaz1973
Updating Dhina1940
Updating Delaporte1839
Updating Landberg1905
Updating Dekkak1979
Updating Décobert1985
Updating Dav

Updating Herrero_Muñoz-Cobo1994
Updating Knüppel2018
Updating D_Anna2018a
Updating Shachmon2018
Updating Bar-Moshe2018
Updating Hocini2010
Updating Klimiuk2017
Updating Grigore2011
Updating McNeil2017
Updating Emerson1943
Updating Stace1893
Updating Jastrow2018a
Updating Rosenhouse2018
Updating Lucas2018
Updating Heath2018
Updating Younes2018b
Updating Dawod1952
Updating Al_Yaari2012
Updating Feghali1990
Updating Qaddūrī1983
Updating Cantineau1940b
Updating Holmes1955
Updating Kuwait_Oil_Company1951
Updating Al-Rūmī1984
Updating Al-Rūmī1982
Updating Al-Rūmī1980
Updating Al-Rūmī1978
Updating al-Nūrī1981
Updating Al-Maghribī1986
Updating Khalaf1989
Updating Khalaf1988
Updating Muḥammad2004
Updating al-Ghunaim2004
Updating as-Sabʿān1989
Updating Al-Rūmī2005
Updating Muḥammad2009
Updating Yassin1977b
Updating Johnstone1964b
Updating Ayyūb_Ḥ.1997
Updating Ayyūb_Ḥ.1982
Updating Solaÿmân1972
Updating Hallman2011
Updating Hallman2016b
Updating Hallman2017
Updating Ambros1982
Updating Behnstedt

Updating Zlitni2015
Updating Mejri2009
Updating Dallaji2014
Updating Ritt-Benmimoun2014c
Updating McNeil2019
Updating Chaâbane1996b
Updating Dallaji2017c
Updating Mion2017b
Updating Mion2013
Updating Al-Khatib1988
Updating Leitner2021a
Updating Khan2019
Updating Lentin1997
Updating Roset2018
Updating Procházka2008c
Updating Nevo1991
Updating Vicente2008e
Updating Gago_Gómez2018
Updating Benítez_Fernández2014
Updating Jastrow2018b
Updating Al-Nassir1993
Updating Edzard2009
Updating Sayahi2014
Updating Payne1983
Updating Biţună2013
Updating Biţună2012
Updating Biţună2011
Updating Maamouri1967
Updating Walters2003
Updating Walters1989
Updating Arnold2017
Updating Hanitsch2019
Updating Procházka2020a
Updating Wilmsen2012
Updating Canna2012
Updating Soltan2012
Updating Bassiouney2012
Updating Ouwaydah2012
Updating Chatar-Moumi2012
Updating Faust2019
Updating Haddad2019
Updating Albaty2018
Updating Laks2018
Updating Slimani2018
Updating Hallman2018c
Updating Sadler2018
Updating Kagan2016
Upd

Updating Abu-Melhim1991
Updating Walters
error to generate id: Walters
Updating Shi_ri2011
Updating Piamenta1990b
Updating Piamenta1991b
Updating Masliyah2010
Updating Halloun1991
Updating Bauer1910
Updating Sieny1978
Updating Behnstedt1985d
Updating D_Anna2016
Updating Benkato2017
Updating Benkato2016
Updating Hoigilt2017a
Updating Hoigilt2017b
Updating Hoigilt2017c
Updating Hoigilt2017d
Updating Hoigilt2017e
Updating Hoigilt2017f
Updating Hoigilt2017g
Updating Hoigilt2017h
Updating Hoigilt2017i
Updating Hoigilt2017j
Updating Hoigilt2017k
Updating Hoigilt2017l
Updating Ferrando2016
Updating Yassin1988
Updating Bouchhioua2017
Updating Belnap1999
Updating Leitner2021b
Updating Leitner2020
Updating Al-Azraqi2010
Updating Hoigilt2017n
Updating Mahdi2010
Updating Bakir2019
Updating Alqassas2019b
Updating Al-Deaibes2019
Updating Laks2019
Updating Moustaoui2019
Updating Tsz-Cheung_Leung2020
Updating Bassiouney2020
Updating Abu_Guba2020b
Updating Abdel_Salam_El_Dakhs2019
Updating Al-Jallad202

In [ ]:
missing[4]

In [ ]:
missing[0]